#### 1. Isoseqsim - pb

In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import mean_squared_error

# Define the base directory where your files are located
base_dir = "terra_outputs/1.isoseqsim_pb"
benchmarking_results_dir = "benchmarking_results/1.isoseqsim_pb"

# Ensure the benchmarking results directory exists
os.makedirs(benchmarking_results_dir, exist_ok=True)

# Define the folders to process
folders = [
    "arabidopsis_isoseqsim_e000", "arabidopsis_isoseqsim_e016", "arabidopsis_isoseqsim_e050", "arabidopsis_isoseqsim_e085",
    "magnaporthe_isoseqsim_e000", "magnaporthe_isoseqsim_e016", "magnaporthe_isoseqsim_e050", "magnaporthe_isoseqsim_e085",
    "mouse_isoseqsim_e000", "mouse_isoseqsim_e016", "mouse_isoseqsim_e050", "mouse_isoseqsim_e085"
]

# Define the file names, methods, transcript columns, and count columns
files = [
    'Bambu_quant.txt', 'Flair_quant.tsv', 'Isosceles_quant.txt', 'IsoQuant_quant.tsv', 
    'Oarfish_quant.tsv', 'Salmon_quant.sf', 'StringTie_quant.csv', 
    'LRAA_0216.quant-only.quant.expr', 'LRAA_0223.quant-only.quant.expr', 'Gffcompare_OUT_expression_matrix.tsv'
]

methods = [
    'bambu', 'flair', 'isosceles', 'isoquant',  
    'oarfish', 'salmon', 'stringtie', 
    'lraa_0216', 'lraa_0223', 'gffcompare'
]

transcript_cols = [
    'txname', 'ids', 'transcript_id', '#feature_id', 'tname', 
    'name', 'transcript_id', 'transcript_id', 
    'transcript_id', 'transcript_id', 'ref_id'
]

count_cols = [
    'count', 'flair_condition1_batch1', 'count', 'count', 'num_reads', 
    'numreads', 'stringtie', 'all_reads', 
    'all_reads', 'all_reads', 'expression'
]

# Define the ground truth file paths
groundtruth_files = {
    'arabidopsis': 'references/1.isoseqsim_pb/Arabidopsis/Arabidopsis_transcript_id_count.tsv',
    'magnaporthe': 'references/1.isoseqsim_pb/Magnaporthe/Magnaporthe_isoseqsim_ground_truth_transcript_id_count.tsv',
    'mouse': 'references/1.isoseqsim_pb/Mouse/Mouse_isoseqsim_ground_truth_transcript_id_count.tsv'
}

# Function to read in a file and add a method column
def read_file(file_name, method_name, transcript_col, count_col):
    delim = ',' if file_name.endswith('.csv') else '\t'
    df = pd.read_csv(file_name, delimiter=delim)
    df.columns = df.columns.str.lower()  # Convert column names to lowercase
    transcript_col = transcript_col.lower()
    count_col = count_col.lower()
    
    if method_name == 'bambu' or count_col not in df.columns:
        if len(df.columns) > 2 or (method_name == 'gffcompare' and 'ref_id' in df.columns and 'expression' in df.columns):
            count_col = 'expression' if method_name == 'gffcompare' else df.columns[2]  # Select the appropriate column
            transcript_col = 'ref_id' if method_name == 'gffcompare' else transcript_col
        else:
            print(f"File {file_name} does not have enough columns. Available columns: {df.columns}")
            return None
    try:
        df = df[[transcript_col, count_col]].rename(columns={transcript_col: 'transcript_name', count_col: 'count'})
    except KeyError:
        print(f"Columns {transcript_col} and {count_col} not found in {file_name}. Available columns: {df.columns}")
        return None
    df['method'] = method_name
    return df

# Function to calculate RMSE
def calculate_rmse(predicted, actual):
    return np.sqrt(mean_squared_error(actual, predicted))

# Function to calculate Mean Relative Difference
def calculate_mrd(predicted, actual):
    return np.mean(np.abs(predicted - actual) / actual)

# Initialize lists to collect all metrics results and merged counts
all_metrics_results = []
all_raw_counts = []

# Process each folder
for folder in folders:
    quant_dir = os.path.join(base_dir, folder, "Quant")
    if not os.path.exists(quant_dir):
        continue
    
    # Determine the ground truth file based on the folder
    if 'arabidopsis' in folder:
        groundtruth_path = groundtruth_files['arabidopsis']
    elif 'magnaporthe' in folder:
        groundtruth_path = groundtruth_files['magnaporthe']
    elif 'mouse' in folder:
        groundtruth_path = groundtruth_files['mouse']
    else:
        continue
    
    # Read the ground truth file
    groundtruth = pd.read_csv(groundtruth_path, delimiter='\t')
    groundtruth = groundtruth[['transcript_id', 'count']].rename(columns={'transcript_id': 'transcript_name'})
    groundtruth['method'] = 'groundtruth'
    
    # Read all quantification files
    data_frames = []
    for file, method, transcript_col, count_col in zip(files, methods, transcript_cols, count_cols):
        file_path = os.path.join(quant_dir, file)
        if os.path.exists(file_path):
            df = read_file(file_path, method, transcript_col, count_col)
            if df is not None:
                data_frames.append(df)
    
    # Combine all data frames into one
    if data_frames:
        data = pd.concat(data_frames)
        all_data = pd.concat([groundtruth, data])
        all_data.fillna(0, inplace=True)
        
        # Ensure the count column contains only numeric values
        all_data['count'] = pd.to_numeric(all_data['count'], errors='coerce').fillna(0)
        
        # Store raw counts for combined_merged_counts.tsv
        raw_data = all_data.pivot(index='transcript_name', columns='method', values='count').fillna(0)
        raw_data['Folder'] = folder
        all_raw_counts.append(raw_data)
        
        all_data['count'] = np.log2(all_data['count'] + 1)
        
        # Reshape the data to wide format
        all_data_wide = all_data.pivot(index='transcript_name', columns='method', values='count')
        all_data_wide = all_data_wide[all_data_wide['groundtruth'].notna()]
        
        merged_data = all_data_wide.fillna(0)
        
        # Calculate metrics for each column against groundtruth, excluding non-numeric columns
        for column_name in merged_data.columns:
            if column_name != 'groundtruth':
                rmse_value = calculate_rmse(merged_data[column_name], merged_data['groundtruth'])
                pearson_value, _ = pearsonr(merged_data[column_name], merged_data['groundtruth'])
                spearman_value, _ = spearmanr(merged_data[column_name], merged_data['groundtruth'])
                mrd_value = calculate_mrd(merged_data[column_name], merged_data['groundtruth'])
                all_metrics_results.append({
                    'Folder': folder,
                    'Tool': column_name,
                    'RMSE': rmse_value,
                    'Pearson_Correlation': pearson_value,
                    'Spearman_Correlation': spearman_value,
                    'Mean_Relative_Difference': mrd_value
                })

# Convert the collected metrics results to a DataFrame
all_metrics_results_df = pd.DataFrame(all_metrics_results)

# Save the combined metrics results to a TSV file
combined_metrics_results_path = os.path.join(benchmarking_results_dir, 'combined_quantification_metrics_results.tsv')
all_metrics_results_df.to_csv(combined_metrics_results_path, sep='\t', index=False)

# Combine all raw counts into a single DataFrame
all_raw_counts_df = pd.concat(all_raw_counts)

# Save the combined raw counts to a TSV file
combined_merged_counts_path = os.path.join(benchmarking_results_dir, 'combined_quantification_merged_counts.tsv')
all_raw_counts_df.to_csv(combined_merged_counts_path, sep='\t')

# Print the combined metrics results
print("Combined Metrics Results:")
print(all_metrics_results_df)

# Generate PDF with plots
pdf_path = os.path.join(benchmarking_results_dir, 'combined_quantification_metrics_plots.pdf')
with PdfPages(pdf_path) as pdf:
    for folder in folders:
        folder_data = all_metrics_results_df[all_metrics_results_df['Folder'] == folder]
        
        fig, axs = plt.subplots(2, 2, figsize=(10, 10))
        fig.suptitle(f'Metrics for {folder}', fontsize=16)
        
        # Sort data for plotting
        folder_data_rmse = folder_data.sort_values(by='RMSE')
        folder_data_mrd = folder_data.sort_values(by='Mean_Relative_Difference')
        folder_data_pearson = folder_data.sort_values(by='Pearson_Correlation', ascending=False)
        folder_data_spearman = folder_data.sort_values(by='Spearman_Correlation', ascending=False)
        
        # RMSE plot
        axs[1, 0].bar(folder_data_rmse['Tool'], folder_data_rmse['RMSE'])
        axs[1, 0].set_title('RMSE')
        axs[1, 0].set_xticks(range(len(folder_data_rmse['Tool'])))
        axs[1, 0].set_xticklabels(folder_data_rmse['Tool'], rotation=45, ha='right')
        
        # Pearson Correlation plot
        axs[0, 0].bar(folder_data_pearson['Tool'], folder_data_pearson['Pearson_Correlation'])
        axs[0, 0].set_title('Pearson Correlation')
        axs[0, 0].set_xticks(range(len(folder_data_pearson['Tool'])))
        axs[0, 0].set_xticklabels(folder_data_pearson['Tool'], rotation=45, ha='right')
        
        # Spearman Correlation plot
        axs[0, 1].bar(folder_data_spearman['Tool'], folder_data_spearman['Spearman_Correlation'])
        axs[0, 1].set_title('Spearman Correlation')
        axs[0, 1].set_xticks(range(len(folder_data_spearman['Tool'])))
        axs[0, 1].set_xticklabels(folder_data_spearman['Tool'], rotation=45, ha='right')
        
        # Mean Relative Difference plot
        axs[1, 1].bar(folder_data_mrd['Tool'], folder_data_mrd['Mean_Relative_Difference'])
        axs[1, 1].set_title('Mean Relative Difference')
        axs[1, 1].set_xticks(range(len(folder_data_mrd['Tool'])))
        axs[1, 1].set_xticklabels(folder_data_mrd['Tool'], rotation=45, ha='right')
        
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        pdf.savefig(fig)
        plt.close(fig)

print(f"PDF with plots saved to {pdf_path}")

Combined Metrics Results:
                         Folder        Tool      RMSE  Pearson_Correlation  \
0    arabidopsis_isoseqsim_e000       bambu  1.440826             0.880243   
1    arabidopsis_isoseqsim_e000       flair  0.960735             0.943238   
2    arabidopsis_isoseqsim_e000  gffcompare  0.893090             0.949031   
3    arabidopsis_isoseqsim_e000    isoquant  0.957517             0.943422   
4    arabidopsis_isoseqsim_e000   isosceles  0.740789             0.964548   
..                          ...         ...       ...                  ...   
115        mouse_isoseqsim_e085   lraa_0216  5.157930             0.054784   
116        mouse_isoseqsim_e085   lraa_0223  5.157930             0.054784   
117        mouse_isoseqsim_e085     oarfish  0.553363             0.978575   
118        mouse_isoseqsim_e085      salmon  1.925620             0.830886   
119        mouse_isoseqsim_e085   stringtie  4.248159             0.686186   

     Spearman_Correlation  Mean_Relat

#### SIRVs - pb

In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import mean_squared_error

# Define the base directory where your files are located
base_dir = "terra_outputs/2.sirvs_pb"
benchmarking_results_dir = "benchmarking_results/2.sirvs_pb"

# Ensure the benchmarking results directory exists
os.makedirs(benchmarking_results_dir, exist_ok=True)

# Define the folders to process
folders = [
    "CL_BT474_E0_sirv", "CL_BT474_E1_sirv", "CL_BT474_E2_sirv",
    "CL_HG002_E0_sirv", "CL_HG002_E1_sirv", "CL_HG002_E2_sirv",
    "CL_K562_E0_sirv", "CL_K562_E1_sirv", "CL_K562_E2_sirv",
    "CL_UHRR_E0_sirv", "CL_UHRR_E1_sirv", "CL_UHRR_E2_sirv"
]

# Define the file names, methods, transcript columns, and count columns
files = [
    'Bambu_quant.txt', 'Flair_quant.tsv', 'Isosceles_quant.txt', 'IsoQuant_quant.tsv', 
    'Oarfish_quant.tsv', 'Salmon_quant.sf', 'StringTie_quant.csv', 
    'LRAA_0223.quant-only.quant.expr', 
    'Gffcompare_OUT_expression_matrix.tsv'
]

methods = [
    'bambu', 'flair', 'isosceles', 'isoquant', 
    'oarfish', 'salmon', 'stringtie', 
    'lraa_0223', 
    'gffcompare'
]

transcript_cols = [
    'txname', 'ids', 'transcript_id', '#feature_id', 'tname', 
    'name', 'transcript_id', 'transcript_id', 
    'transcript_id', 
    'ref_id'
]

count_cols = [
    '', 'flair_condition1_batch1', 'count', 'count', 'num_reads', 
    'numreads', 'stringtie', 'all_reads', 
    'all_reads', 
    'expression'
]

# Define the ground truth file paths
groundtruth_files = {
    'CL_BT474_E0_sirv': 'BT474_E0_merged_sirv_sorted_groundtruth_E0.tsv',
    'CL_BT474_E1_sirv': 'BT474_E1_merged_sirv_sorted_groundtruth_E1.tsv',
    'CL_BT474_E2_sirv': 'BT474_E2_merged_sirv_sorted_groundtruth_E2.tsv',
    'CL_HG002_E0_sirv': 'HG002_E0_merged_sirv_sorted_groundtruth_E0.tsv',
    'CL_HG002_E1_sirv': 'HG002_E1_merged_sirv_sorted_groundtruth_E1.tsv',
    'CL_HG002_E2_sirv': 'HG002_E2_merged_sirv_sorted_groundtruth_E2.tsv',
    'CL_K562_E0_sirv': 'K562_E0_merged_sirv_sorted_groundtruth_E0.tsv',
    'CL_K562_E1_sirv': 'K562_E1_merged_sirv_sorted_groundtruth_E1.tsv',
    'CL_K562_E2_sirv': 'K562_E2_merged_sirv_sorted_groundtruth_E2.tsv',
    'CL_UHRR_E0_sirv': 'UHRR_E0_merged_sirv_sorted_groundtruth_E0.tsv',
    'CL_UHRR_E1_sirv': 'UHRR_E1_merged_sirv_sorted_groundtruth_E1.tsv',
    'CL_UHRR_E2_sirv': 'UHRR_E2_merged_sirv_sorted_groundtruth_E2.tsv'
}

groundtruth_base_dir = 'references/2.sirvs_pb/quantification_ground_truth'

# Function to read in a file and add a method column
def read_file(file_name, method_name, transcript_col, count_col):
    delim = ',' if file_name.endswith('.csv') else '\t'
    df = pd.read_csv(file_name, delimiter=delim)
    df.columns = df.columns.str.lower()  # Convert column names to lowercase
    transcript_col = transcript_col.lower()
    count_col = count_col.lower()
    
    # Handle Bambu specific case
    if method_name == 'bambu':
        count_col = df.columns[2]  # Always use the third column for Bambu
    
    # Handle Gffcompare specific case
    if method_name == 'gffcompare':
        transcript_col = 'ref_id'
        count_col = 'expression'
    
    try:
        df = df[[transcript_col, count_col]].rename(columns={transcript_col: 'transcript_name', count_col: 'count'})
    except KeyError:
        print(f"Columns {transcript_col} and {count_col} not found in {file_name}. Available columns: {df.columns}")
        return None
    df['method'] = method_name
    return df

# Function to calculate RMSE
def calculate_rmse(predicted, actual):
    return np.sqrt(mean_squared_error(actual, predicted))

# Function to calculate Mean Relative Difference
def calculate_mrd(predicted, actual):
    return np.mean(np.abs(predicted - actual) / actual)

# Initialize lists to collect all metrics results and merged counts
all_metrics_results = []
all_raw_counts = []

# Process each folder
for folder in folders:
    quant_dir = os.path.join(base_dir, folder, "Quant")
    if not os.path.exists(quant_dir):
        continue
    
    # Determine the ground truth file based on the folder
    groundtruth_file = groundtruth_files[folder]
    groundtruth_path = os.path.join(groundtruth_base_dir, groundtruth_file)
    
    # Read the ground truth file
    groundtruth = pd.read_csv(groundtruth_path, delimiter='\t')
    groundtruth = groundtruth[['transcript_id', 'count']].rename(columns={'transcript_id': 'transcript_name'})
    groundtruth['method'] = 'groundtruth'
    
    # Read all quantification files
    data_frames = []
    for file, method, transcript_col, count_col in zip(files, methods, transcript_cols, count_cols):
        file_path = os.path.join(quant_dir, file)
        if os.path.exists(file_path):
            df = read_file(file_path, method, transcript_col, count_col)
            if df is not None:
                data_frames.append(df)
    
    # Combine all data frames into one
    data = pd.concat(data_frames)
    all_data = pd.concat([groundtruth, data])
    all_data.fillna(0, inplace=True)
    
    # Ensure the count column contains only numeric values
    all_data['count'] = pd.to_numeric(all_data['count'], errors='coerce').fillna(0)
    
    # Store raw counts for combined_merged_counts.tsv
    raw_data = all_data.pivot(index='transcript_name', columns='method', values='count').fillna(0)
    raw_data['Folder'] = folder
    all_raw_counts.append(raw_data)
    
    all_data['count'] = np.log2(all_data['count'] + 1)
    
    # Reshape the data to wide format
    all_data_wide = all_data.pivot(index='transcript_name', columns='method', values='count')
    all_data_wide = all_data_wide[all_data_wide['groundtruth'].notna()]
    
    merged_data = all_data_wide.fillna(0)
    
    # Calculate metrics for each column against groundtruth, excluding non-numeric columns
    for column_name in merged_data.columns:
        if column_name != 'groundtruth':
            rmse_value = calculate_rmse(merged_data[column_name], merged_data['groundtruth'])
            if np.all(merged_data[column_name].iloc[:] == merged_data[column_name].iloc[0]):
                pearson_value = spearman_value = np.nan
            else:
                pearson_value, _ = pearsonr(merged_data[column_name], merged_data['groundtruth'])
                spearman_value, _ = spearmanr(merged_data[column_name], merged_data['groundtruth'])
            mrd_value = calculate_mrd(merged_data[column_name], merged_data['groundtruth'])
            all_metrics_results.append({
                'Folder': folder,
                'Tool': column_name,
                'RMSE': rmse_value,
                'Pearson_Correlation': pearson_value,
                'Spearman_Correlation': spearman_value,
                'Mean_Relative_Difference': mrd_value
            })

# Convert the collected metrics results to a DataFrame
all_metrics_results_df = pd.DataFrame(all_metrics_results)

# Save the combined metrics results to a TSV file
combined_metrics_results_path = os.path.join(benchmarking_results_dir, 'combined_quantification_metrics_results.tsv')
all_metrics_results_df.to_csv(combined_metrics_results_path, sep='\t', index=False)

# Combine all raw counts into a single DataFrame
all_raw_counts_df = pd.concat(all_raw_counts)

# Save the combined raw counts to a TSV file
combined_merged_counts_path = os.path.join(benchmarking_results_dir, 'combined_quantification_merged_counts.tsv')
all_raw_counts_df.to_csv(combined_merged_counts_path, sep='\t')

# Print the combined metrics results
print("Combined Metrics Results:")
print(all_metrics_results_df)

# Generate PDF with plots
pdf_path = os.path.join(benchmarking_results_dir, 'combined_quantification_metrics_plots.pdf')
with PdfPages(pdf_path) as pdf:
    for folder in folders:
        folder_data = all_metrics_results_df[all_metrics_results_df['Folder'] == folder]
        
        fig, axs = plt.subplots(2, 2, figsize=(10, 10))
        fig.suptitle(f'Metrics for {folder}', fontsize=16)
        
        # Sort data for plotting
        folder_data_rmse = folder_data.sort_values(by='RMSE')
        folder_data_mrd = folder_data.sort_values(by='Mean_Relative_Difference')
        folder_data_pearson = folder_data.sort_values(by='Pearson_Correlation', ascending=False)
        folder_data_spearman = folder_data.sort_values(by='Spearman_Correlation', ascending=False)
        
        # RMSE plot
        axs[1, 0].bar(folder_data_rmse['Tool'], folder_data_rmse['RMSE'])
        axs[1, 0].set_title('RMSE')
        axs[1, 0].set_xticks(range(len(folder_data_rmse['Tool'])))
        axs[1, 0].set_xticklabels(folder_data_rmse['Tool'], rotation=45, ha='right')
        
        # Pearson Correlation plot
        axs[0, 0].bar(folder_data_pearson['Tool'], folder_data_pearson['Pearson_Correlation'])
        axs[0, 0].set_title('Pearson Correlation')
        axs[0, 0].set_xticks(range(len(folder_data_pearson['Tool'])))
        axs[0, 0].set_xticklabels(folder_data_pearson['Tool'], rotation=45, ha='right')
        
        # Spearman Correlation plot
        axs[0, 1].bar(folder_data_spearman['Tool'], folder_data_spearman['Spearman_Correlation'])
        axs[0, 1].set_title('Spearman Correlation')
        axs[0, 1].set_xticks(range(len(folder_data_spearman['Tool'])))
        axs[0, 1].set_xticklabels(folder_data_spearman['Tool'], rotation=45, ha='right')
        
        # Mean Relative Difference plot
        axs[1, 1].bar(folder_data_mrd['Tool'], folder_data_mrd['Mean_Relative_Difference'])
        axs[1, 1].set_title('Mean Relative Difference')
        axs[1, 1].set_xticks(range(len(folder_data_mrd['Tool'])))
        axs[1, 1].set_xticklabels(folder_data_mrd['Tool'], rotation=45, ha='right')
        
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        pdf.savefig(fig)
        plt.close(fig)

print(f"PDF with plots saved to {pdf_path}")

/var/tmp/ipykernel_377369/462205455.py:160: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  pearson_value, _ = pearsonr(merged_data[column_name], merged_data['groundtruth'])
/var/tmp/ipykernel_377369/462205455.py:161: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_value, _ = spearmanr(merged_data[column_name], merged_data['groundtruth'])
/var/tmp/ipykernel_377369/462205455.py:160: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  pearson_value, _ = pearsonr(merged_data[column_name], merged_data['groundtruth'])
/var/tmp/ipykernel_377369/462205455.py:161: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_value, _ = spearmanr(merged_data[column_name], merged_data['groundtruth'])
/var/tmp/ipykernel_377369/462205455.py:160: ConstantInputWarning: An input array is constant; the correlatio

Combined Metrics Results:
               Folder        Tool      RMSE  Pearson_Correlation  \
0    CL_BT474_E0_sirv       bambu  2.802756                  NaN   
1    CL_BT474_E0_sirv       flair  2.699533                  NaN   
2    CL_BT474_E0_sirv  gffcompare  1.429335                  NaN   
3    CL_BT474_E0_sirv    isoquant  2.046609                  NaN   
4    CL_BT474_E0_sirv   isosceles  2.974661                  NaN   
..                ...         ...       ...                  ...   
103   CL_UHRR_E2_sirv   isosceles  3.221857             0.574289   
104   CL_UHRR_E2_sirv   lraa_0223  1.315622             0.858178   
105   CL_UHRR_E2_sirv     oarfish  1.793809             0.749296   
106   CL_UHRR_E2_sirv      salmon  3.083520             0.444839   
107   CL_UHRR_E2_sirv   stringtie  5.636042             0.371968   

     Spearman_Correlation  Mean_Relative_Difference  
0                     NaN                  0.126136  
1                     NaN                  0.1241

#### MORFs

In [11]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import mean_squared_error

# Define the base directory where your files are located
base_dir = "terra_outputs/3.morfs_pb_ont"
benchmarking_results_dir = "benchmarking_results/3.morfs_pb_ont"

# Ensure the benchmarking results directory exists
os.makedirs(benchmarking_results_dir, exist_ok=True)

# Define the folders to process
folders = [
    "morf2_ont_merged_annot_compat_1isoform", 
    "morf2_pacbio_merged_annot_compat_1isoform"
]

# Define the file names, methods, transcript columns, and count columns
files = [
    'Bambu_quant.txt', 'Flair_quant.tsv', 'Isosceles_quant.txt', 'IsoQuant_quant.tsv', 
    'Oarfish_quant.tsv', 'Salmon_quant.sf', 'StringTie_quant.csv', 
    'LRAA_0223.quant-only.quant.expr', 
    'Gffcompare_OUT_expression_matrix.tsv'
]

methods = [
    'bambu', 'flair', 'isosceles', 'isoquant', 
    'oarfish', 'salmon', 'stringtie', 
    'lraa_0223', 
    'gffcompare'
]

transcript_cols = [
    'txname', 'ids', 'transcript_id', '#feature_id', 'tname', 
    'name', 'transcript_id', 'transcript_id', 
    'transcript_id', 
    'ref_id'
]

count_cols = [
    '', 'flair_condition1_batch1', 'count', 'count', 'num_reads', 
    'numreads', 'stringtie', 'all_reads', 
    'all_reads', 
    'expression'
]

# Define the ground truth file paths
groundtruth_files = {
    'morf2_ont_merged_annot_compat_1isoform': 'references/3.morfs_pb_ont/ont/morf2_ont_merged_annot_compat_sorted_tn_counts.tsv',
    'morf2_pacbio_merged_annot_compat_1isoform': 'references/3.morfs_pb_ont/pb/morf2_pacbio_merged_annot_compat_sorted_tn_counts.tsv'
}

# Function to read in a file and add a method column
def read_file(file_name, method_name, transcript_col, count_col):
    delim = ',' if file_name.endswith('.csv') else '\t'
    df = pd.read_csv(file_name, delimiter=delim)
    df.columns = df.columns.str.lower()  # Convert column names to lowercase
    transcript_col = transcript_col.lower()
    count_col = count_col.lower()
    
    # Handle Bambu specific case
    if method_name == 'bambu':
        count_col = df.columns[2]  # Always use the third column for Bambu
    
    # Handle Gffcompare specific case
    if method_name == 'gffcompare':
        transcript_col = 'ref_id'
        count_col = 'expression'
    
    try:
        df = df[[transcript_col, count_col]].rename(columns={transcript_col: 'transcript_name', count_col: 'count'})
    except KeyError:
        print(f"Columns {transcript_col} and {count_col} not found in {file_name}. Available columns: {df.columns}")
        return None
    df['method'] = method_name
    return df

# Function to calculate RMSE
def calculate_rmse(predicted, actual):
    return np.sqrt(mean_squared_error(actual, predicted))

# Function to calculate Mean Relative Difference
def calculate_mrd(predicted, actual):
    return np.mean(np.abs(predicted - actual) / actual)

# Initialize lists to collect all metrics results and merged counts
all_metrics_results = []
all_raw_counts = []

# Process each folder
for folder in folders:
    quant_dir = os.path.join(base_dir, folder, "Quant")
    if not os.path.exists(quant_dir):
        continue
    
    # Determine the ground truth file based on the folder
    groundtruth_path = groundtruth_files[folder]
    
    # Read the ground truth file
    groundtruth = pd.read_csv(groundtruth_path, delimiter='\t')
    groundtruth = groundtruth[['transcript_id', 'count']].rename(columns={'transcript_id': 'transcript_name'})
    groundtruth['method'] = 'groundtruth'
    
    # Read all quantification files
    data_frames = []
    for file, method, transcript_col, count_col in zip(files, methods, transcript_cols, count_cols):
        file_path = os.path.join(quant_dir, file)
        if os.path.exists(file_path):
            df = read_file(file_path, method, transcript_col, count_col)
            if df is not None:
                data_frames.append(df)
    
    # Combine all data frames into one
    data = pd.concat(data_frames)
    all_data = pd.concat([groundtruth, data])
    all_data.fillna(0, inplace=True)
    
    # Ensure the count column contains only numeric values
    all_data['count'] = pd.to_numeric(all_data['count'], errors='coerce').fillna(0)
    
    # Store raw counts for combined_merged_counts.tsv
    raw_data = all_data.pivot(index='transcript_name', columns='method', values='count').fillna(0)
    raw_data['Folder'] = folder
    all_raw_counts.append(raw_data)
    
    all_data['count'] = np.log2(all_data['count'] + 1)
    
    # Reshape the data to wide format
    all_data_wide = all_data.pivot(index='transcript_name', columns='method', values='count')
    all_data_wide = all_data_wide[all_data_wide['groundtruth'].notna()]
    
    merged_data = all_data_wide.fillna(0)
    
    # Calculate metrics for each column against groundtruth, excluding non-numeric columns
    for column_name in merged_data.columns:
        if column_name != 'groundtruth':
            rmse_value = calculate_rmse(merged_data[column_name], merged_data['groundtruth'])
            if np.all(merged_data[column_name].iloc[:] == merged_data[column_name].iloc[0]):
                pearson_value = spearman_value = np.nan
            else:
                pearson_value, _ = pearsonr(merged_data[column_name], merged_data['groundtruth'])
                spearman_value, _ = spearmanr(merged_data[column_name], merged_data['groundtruth'])
            mrd_value = calculate_mrd(merged_data[column_name], merged_data['groundtruth'])
            all_metrics_results.append({
                'Folder': folder,
                'Tool': column_name,
                'RMSE': rmse_value,
                'Pearson_Correlation': pearson_value,
                'Spearman_Correlation': spearman_value,
                'Mean_Relative_Difference': mrd_value
            })

# Convert the collected metrics results to a DataFrame
all_metrics_results_df = pd.DataFrame(all_metrics_results)

# Save the combined metrics results to a TSV file
combined_metrics_results_path = os.path.join(benchmarking_results_dir, 'combined_quantification_metrics_results.tsv')
all_metrics_results_df.to_csv(combined_metrics_results_path, sep='\t', index=False)

# Combine all raw counts into a single DataFrame
all_raw_counts_df = pd.concat(all_raw_counts)

# Save the combined raw counts to a TSV file
combined_merged_counts_path = os.path.join(benchmarking_results_dir, 'combined_quantification_merged_counts.tsv')
all_raw_counts_df.to_csv(combined_merged_counts_path, sep='\t')

# Print the combined metrics results
print("Combined Metrics Results:")
print(all_metrics_results_df)

# Generate PDF with plots
pdf_path = os.path.join(benchmarking_results_dir, 'combined_quantification_metrics_plots.pdf')
with PdfPages(pdf_path) as pdf:
    for folder in folders:
        folder_data = all_metrics_results_df[all_metrics_results_df['Folder'] == folder]
        
        fig, axs = plt.subplots(2, 2, figsize=(10, 10))
        fig.suptitle(f'Metrics for {folder}', fontsize=16)
        
        # Sort data for plotting
        folder_data_rmse = folder_data.sort_values(by='RMSE')
        folder_data_mrd = folder_data.sort_values(by='Mean_Relative_Difference')
        folder_data_pearson = folder_data.sort_values(by='Pearson_Correlation', ascending=False)
        folder_data_spearman = folder_data.sort_values(by='Spearman_Correlation', ascending=False)
        
        # RMSE plot
        axs[1, 0].bar(folder_data_rmse['Tool'], folder_data_rmse['RMSE'])
        axs[1, 0].set_title('RMSE')
        axs[1, 0].set_xticks(range(len(folder_data_rmse['Tool'])))
        axs[1, 0].set_xticklabels(folder_data_rmse['Tool'], rotation=45, ha='right')
        
        # Pearson Correlation plot
        axs[0, 0].bar(folder_data_pearson['Tool'], folder_data_pearson['Pearson_Correlation'])
        axs[0, 0].set_title('Pearson Correlation')
        axs[0, 0].set_xticks(range(len(folder_data_pearson['Tool'])))
        axs[0, 0].set_xticklabels(folder_data_pearson['Tool'], rotation=45, ha='right')
        
        # Spearman Correlation plot
        axs[0, 1].bar(folder_data_spearman['Tool'], folder_data_spearman['Spearman_Correlation'])
        axs[0, 1].set_title('Spearman Correlation')
        axs[0, 1].set_xticks(range(len(folder_data_spearman['Tool'])))
        axs[0, 1].set_xticklabels(folder_data_spearman['Tool'], rotation=45, ha='right')
        
        # Mean Relative Difference plot
        axs[1, 1].bar(folder_data_mrd['Tool'], folder_data_mrd['Mean_Relative_Difference'])
        axs[1, 1].set_title('Mean Relative Difference')
        axs[1, 1].set_xticks(range(len(folder_data_mrd['Tool'])))
        axs[1, 1].set_xticklabels(folder_data_mrd['Tool'], rotation=45, ha='right')
        
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        pdf.savefig(fig)
        plt.close(fig)

print(f"PDF with plots saved to {pdf_path}")

Combined Metrics Results:
                                       Folder        Tool      RMSE  \
0      morf2_ont_merged_annot_compat_1isoform       bambu  1.944429   
1      morf2_ont_merged_annot_compat_1isoform       flair  2.270548   
2      morf2_ont_merged_annot_compat_1isoform  gffcompare  1.805018   
3      morf2_ont_merged_annot_compat_1isoform    isoquant  2.732091   
4      morf2_ont_merged_annot_compat_1isoform   isosceles  1.630663   
5      morf2_ont_merged_annot_compat_1isoform   lraa_0223  0.641843   
6      morf2_ont_merged_annot_compat_1isoform     oarfish  0.702956   
7      morf2_ont_merged_annot_compat_1isoform      salmon  1.787564   
8      morf2_ont_merged_annot_compat_1isoform   stringtie  4.961009   
9   morf2_pacbio_merged_annot_compat_1isoform       bambu  2.341029   
10  morf2_pacbio_merged_annot_compat_1isoform       flair  3.019903   
11  morf2_pacbio_merged_annot_compat_1isoform  gffcompare  2.116623   
12  morf2_pacbio_merged_annot_compat_1isoform    is